### Setup

In [1]:
import keras_cv

Using TensorFlow backend


C:\Users\tristan_cotte\PycharmProjects\yolov8_keras\.venv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
import os
from tqdm.auto import tqdm
import xml.etree.ElementTree as ET

import tensorflow as tf
from tensorflow import keras

import keras_cv
from keras_cv import bounding_box
from keras_cv import visualization


In [ ]:
# !pip install --upgrade git+https://github.com/keras-team/keras-cv -q


In [2]:
SPLIT_RATIO = 0.2
BATCH_SIZE = 4
LEARNING_RATE = 0.001
EPOCH = 5
GLOBAL_CLIPNORM = 10.0


In [141]:
class_ids = [
    "Model",
    "Added",
]
class_mapping = dict(zip(range(len(class_ids)), class_ids))

# Path to images and annotations
path_images = "../dataset_pascal_voc/Evry_dataset_2024_pascal_voc/VOCdevkit/VOC/JPEGImages"
path_annot = "../dataset_pascal_voc/Evry_dataset_2024_pascal_voc/VOCdevkit/VOC/Annotations"

# Get all XML file paths in path_annot and sort them
xml_files = sorted(
    [
        os.path.join(path_annot, file_name)
        for file_name in os.listdir(path_annot)
        if file_name.endswith(".xml")
    ]
)

# Get all JPEG image file paths in path_images and sort them
jpg_files = sorted(
    [
        os.path.join(path_images, file_name)
        for file_name in os.listdir(path_images)
        if file_name.endswith(".jpg")
    ]
)


In [142]:
def parse_annotation(xml_file, single_cls: bool=False):
    tree = ET.parse(xml_file)
    root = tree.getroot()

    image_name = root.find("filename").text
    image_path = os.path.join(path_images, image_name)

    boxes = []
    classes = []
    for obj in root.iter("object"):
        cls = obj.find("name").text
        classes.append(cls)

        bbox = obj.find("bndbox")
        xmin = float(bbox.find("xmin").text)
        ymin = float(bbox.find("ymin").text)
        xmax = float(bbox.find("xmax").text)
        ymax = float(bbox.find("ymax").text)
        boxes.append([xmin, ymin, xmax, ymax])
    
    if not single_cls:
        class_ids = [
            list(class_mapping.keys())[list(class_mapping.values()).index(cls)]
            for cls in classes
        ]
    else:
        class_ids = [0]*len(classes)
    return image_path, boxes, class_ids


image_paths = []
bbox = []
classes = []
for xml_file in tqdm(xml_files):
    image_path, boxes, class_ids = parse_annotation(xml_file, single_cls=True)
    image_paths.append(image_path)
    bbox.append(boxes)
    classes.append(class_ids)


100%|██████████| 928/928 [00:02<00:00, 400.92it/s] 


In [143]:
for xml_file in tqdm(xml_files):
    tree = ET.parse(xml_file)
    root = tree.getroot()

    image_name = root.find("filename").text
    if image_name.endswith('.png'):
        print(image_name)
        root.find("filename").text = image_name.replace('.png', '.jpg')
        tree.write(xml_file)
        

100%|██████████| 928/928 [00:01<00:00, 529.30it/s] 


In [144]:
bbox = tf.ragged.constant(bbox)
classes = tf.ragged.constant(classes)
image_paths = tf.ragged.constant(image_paths)

data = tf.data.Dataset.from_tensor_slices((image_paths, classes, bbox))


In [145]:
# Determine the number of validation samples
num_val = int(len(xml_files) * SPLIT_RATIO)

# Split the dataset into train and validation sets
val_data = data.take(num_val)
train_data = data.skip(num_val)


In [146]:
def load_image(image_path):
    image = tf.io.read_file(image_path)
    image = tf.image.decode_jpeg(image, channels=3)
    return image


def load_dataset(image_path, classes, bbox):
    # Read Image
    image = load_image(image_path)
    bounding_boxes = {
        "classes": tf.cast(classes, dtype=tf.float32),
        "boxes": bbox,
    }
    return {"images": tf.cast(image, tf.float32), "bounding_boxes": bounding_boxes}


### Data augmentation

In [18]:
# !pip install tensorflow_datasets

  Using cached tensorflow_datasets-4.9.3-py3-none-any.whl.metadata (9.3 kB)
Using cached tensorflow_datasets-4.9.3-py3-none-any.whl (5.0 MB)


In [126]:
# os.environ["KERAS_BACKEND"] = "tensorflow"
import keras_cv
# import keras_core as keras

In [147]:
augmenter = keras.Sequential(
    layers=[
        keras_cv.layers.RandomFlip(mode="horizontal", bounding_box_format="xyxy"),
        keras_cv.layers.RandomShear(
            x_factor=0.2, y_factor=0.2, bounding_box_format="xyxy"
        ),
        keras_cv.layers.JitteredResize(
            target_size=(640, 640), scale_factor=(0.75, 1.3), bounding_box_format="xyxy"
        ),
    ]
)


### Create training and validation datasets

In [148]:
train_ds = train_data.map(load_dataset, num_parallel_calls=tf.data.AUTOTUNE)
train_ds = train_ds.shuffle(BATCH_SIZE * 4)
train_ds = train_ds.ragged_batch(BATCH_SIZE, drop_remainder=True)
train_ds = train_ds.map(augmenter, num_parallel_calls=tf.data.AUTOTUNE)


In [149]:
resizing = keras_cv.layers.JitteredResize(
    target_size=(640, 640),
    scale_factor=(0.75, 1.3),
    bounding_box_format="xyxy",
)

val_ds = val_data.map(load_dataset, num_parallel_calls=tf.data.AUTOTUNE)
val_ds = val_ds.shuffle(BATCH_SIZE * 4)
val_ds = val_ds.ragged_batch(BATCH_SIZE, drop_remainder=True)
val_ds = val_ds.map(resizing, num_parallel_calls=tf.data.AUTOTUNE)


### Visualization

In [150]:
def visualize_dataset(inputs, value_range, rows, cols, bounding_box_format):
    inputs = next(iter(inputs.take(1)))
    images, bounding_boxes = inputs["images"], inputs["bounding_boxes"]
    visualization.plot_bounding_box_gallery(
        images,
        value_range=value_range,
        rows=rows,
        cols=cols,
        y_true=bounding_boxes,
        scale=5,
        font_scale=0.7,
        bounding_box_format=bounding_box_format,
        class_mapping=class_mapping,
    )


visualize_dataset(
    train_ds, bounding_box_format="xyxy", value_range=(0, 255), rows=2, cols=2
)

visualize_dataset(
    val_ds, bounding_box_format="xyxy", value_range=(0, 255), rows=2, cols=2
)


ImportError: plot_bounding_box_gallery requires the `matplotlib` package. Please install the package using `pip install matplotlib`.

In [151]:

def dict_to_tuple(inputs):
    # https://github.com/keras-team/keras-io/issues/1475#issuecomment-1716460270
    return inputs["images"], bounding_box.to_dense(
        inputs["bounding_boxes"], max_boxes=1000
    )

train_ds = train_ds.map(dict_to_tuple, num_parallel_calls=tf.data.AUTOTUNE)
train_ds = train_ds.prefetch(tf.data.AUTOTUNE)

val_ds = val_ds.map(dict_to_tuple, num_parallel_calls=tf.data.AUTOTUNE)
val_ds = val_ds.prefetch(tf.data.AUTOTUNE)


### Create model

In [21]:
os.environ["HTTP_PROXY"] = "http://127.0.0.1:9000/localproxy-68e785bc.pac"

In [41]:
import requests
r = requests.get('https://www.kaggle.com', verify=r"C:\Users\tristan_cotte\PycharmProjects\yolov8_keras\certificates\Zscaler Root CA.crt")

In [63]:
import urllib3
from urllib3.connectionpool import HTTPSConnectionPool


conn = HTTPSConnectionPool(host='www.kaggle.com', port=443, cert_file=r"C:\Users\tristan_cotte\PycharmProjects\yolov8_keras\certificates\Zscaler Root CA.crt")

In [56]:
!export PYTHONHTTPSVERIFY=0 

'export' is not recognized as an internal or external command,
operable program or batch file.


In [60]:
import ssl

ssl.SSLContext.verify_mode = ssl.VerifyMode.CERT_OPTIONAL

In [65]:

http = urllib3.PoolManager(
    cert_reqs="CERT_REQUIRED",
    ca_certs=r"C:\Users\tristan_cotte\PycharmProjects\yolov8_keras\certificates\Zscaler Root CA.crt"
)

In [69]:
import requests
r = requests.get('https://www.kaggle.com', verify=r"C:\Users\tristan_cotte\PycharmProjects\yolov8_keras\certificates\Zscaler Root CA.crt")

backbone = keras_cv.models.YOLOV8Backbone.from_preset(
    "yolo_v8_xs_backbone_coco"  
)

SSLError: HTTPSConnectionPool(host='www.kaggle.com', port=443): Max retries exceeded with url: /api/v1/models/keras/yolov8/keras/yolo_v8_xs_backbone_coco/2/download/config.json (Caused by SSLError(SSLCertVerificationError(1, '[SSL: CERTIFICATE_VERIFY_FAILED] certificate verify failed: unable to get local issuer certificate (_ssl.c:1129)')))

In [27]:

backbone = keras_cv.models.YOLOV8Backbone.from_config()

TypeError: keras_cv.src.models.object_detection.yolo_v8.yolo_v8_backbone.YOLOV8Backbone() argument after ** must be a mapping, not str

In [133]:
import json

config_file = os.path.join(xs_configuration_yolo_folder, "config.json")

with open(config_file, 'r') as JSON:
    json_dict = json.load(JSON)

In [99]:
json_dict['config']

{'name': 'yolov8_backbone',
 'trainable': True,
 'include_rescaling': True,
 'input_shape': [None, None, 3],
 'stackwise_channels': [32, 64, 128, 256],
 'stackwise_depth': [1, 2, 2, 1],
 'activation': 'swish'}

In [134]:
backbone = keras_cv.models.YOLOV8Backbone.from_config(json_dict['config'])

In [95]:
import inspect

def legacy_load_weights(layer, weights_path):
    # Hacky fix for TensorFlow 2.13 and 2.14 when loading a `.weights.h5` file.
    # We find the `Functional` class, and temporarily remove the
    # `_layer_checkpoint_dependencies` property, which on older version of
    # TensorFlow complete broke the variable paths for functional models.
    functional_cls = None
    for cls in inspect.getmro(layer.__class__):
        if cls.__name__ == "Functional":
            functional_cls = cls
    property = functional_cls._layer_checkpoint_dependencies
    functional_cls._layer_checkpoint_dependencies = {}
    layer.load_weights(weights_path)
    functional_cls._layer_checkpoint_dependencies = property


In [101]:
xs_configuration_yolo_folder = r"C:\Users\tristan_cotte\Downloads"

layer=backbone
weights_path = os.path.join(xs_configuration_yolo_folder, 'model.weights.h5')
if hasattr(layer, "_layer_checkpoint_dependencies"):
    legacy_load_weights(layer, weights_path)
else:
    layer.load_weights(weights_path)

ValueError: Layer 'stem_1_conv' expected 1 variables, but received 0 variables during loading. Expected: ['stem_1_conv/kernel:0']

In [102]:


backbone = backbone.load_weights(os.path.join(xs_configuration_yolo_folder, 'model.weights.h5'))

ValueError: Layer 'stem_1_conv' expected 1 variables, but received 0 variables during loading. Expected: ['stem_1_conv/kernel:0']

In [103]:
class_mapping

{0: 'Model', 1: 'Added'}

In [135]:
yolo = keras_cv.models.YOLOV8Detector(
    num_classes=len(class_mapping),
    bounding_box_format="xyxy",
    backbone=backbone,
    fpn_depth=1,
)

In [136]:
optimizer = tf.keras.optimizers.Adam(
    learning_rate=LEARNING_RATE,
    global_clipnorm=GLOBAL_CLIPNORM,
)

yolo.compile(
    optimizer=optimizer, classification_loss="binary_crossentropy", box_loss="ciou",  jit_compile=False
)

In [137]:
class EvaluateCOCOMetricsCallback(keras.callbacks.Callback):
    def __init__(self, data, save_path):
        super().__init__()
        self.data = data
        self.metrics = keras_cv.metrics.BoxCOCOMetrics(
            bounding_box_format="xyxy",
            evaluate_freq=1e9,
        )

        self.save_path = save_path
        self.best_map = -1.0

    def on_epoch_end(self, epoch, logs):
        self.metrics.reset_state()
        for batch in self.data:
            images, y_true = batch[0], batch[1]
            y_pred = self.model.predict(images, verbose=0)
            self.metrics.update_state(y_true, y_pred)

        metrics = self.metrics.result(force=True)
        logs.update(metrics)

        current_map = metrics["MaP"]
        if current_map > self.best_map:
            self.best_map = current_map
            self.model.save(self.save_path)  # Save the model when mAP improves

        return logs

In [152]:
yolo.fit(
    train_ds,
    validation_data=val_ds,
    epochs=3,
    callbacks=[EvaluateCOCOMetricsCallback(val_ds, "model.h5")],
)

Epoch 1/3
185/185 [==============================] - ETA: 0s - loss: 15.2010 - box_loss: 3.6082 - class_loss: 11.5928

UnknownError: {{function_node __wrapped__EagerPyFunc_Tin_1_Tout_1_device_/job:localhost/replica:0/task:0/device:CPU:0}} ImportError: compute_pycoco_metrics requires the `pycocotools` package. Please install the package using `pip install pycocotools`.
Traceback (most recent call last):

  File "C:\Users\tristan_cotte\PycharmProjects\yolov8_keras\.venv\lib\site-packages\tensorflow\python\ops\script_ops.py", line 266, in __call__
    return func(device, token, args)

  File "C:\Users\tristan_cotte\PycharmProjects\yolov8_keras\.venv\lib\site-packages\tensorflow\python\ops\script_ops.py", line 144, in __call__
    outputs = self._call(device, args)

  File "C:\Users\tristan_cotte\PycharmProjects\yolov8_keras\.venv\lib\site-packages\tensorflow\python\ops\script_ops.py", line 151, in _call
    ret = self._func(*args)

  File "C:\Users\tristan_cotte\PycharmProjects\yolov8_keras\.venv\lib\site-packages\tensorflow\python\autograph\impl\api.py", line 643, in wrapper
    return func(*args, **kwar

In [153]:
yolo.evaluate(val_ds, batch_size=4)

46/46 [==============================] - 115s 2s/step - loss: 5.5190 - box_loss: 2.9449 - class_loss: 2.5741


[5.519024848937988, 2.94490647315979, 2.5741193294525146]

In [154]:
physical_device = tf.config.experimental.list_physical_devices('GPU')
print(f'Device found : {physical_device}')

Device found : []


In [155]:
tf.config.experimental.get_memory_growth(physical_device[0])

IndexError: list index out of range

In [159]:
import tensorflow as tf
from tensorflow.python.client import device_lib

print(device_lib.list_local_devices())
print(tf.__version__)

print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))
tf.test.is_gpu_available()

[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 16810409903067121984
xla_global_id: -1
]
2.13.1
Num GPUs Available:  0


False

In [160]:
tf.config.list_physical_devices('GPU')

[]

In [115]:
os.path.isfile(r'../dataset_pascal_voc/Evry_dataset_2024_pascal_voc/VOCdevkit/VOC/JPEGImages\957739_EV22-06386.001-14-03-2022_RAW.jpg')

True

In [113]:
val_ds

<_PrefetchDataset element_spec=(TensorSpec(shape=(4, 640, 640, 3), dtype=tf.float32, name=None), {'classes': RaggedTensorSpec(TensorShape([4, None]), tf.float32, 1, tf.int64), 'boxes': RaggedTensorSpec(TensorShape([4, None, None]), tf.float32, 1, tf.int64)})>